<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/PyTorchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This* NoteBook illustrates the examples given in chapter 1 of Inside Deep Learning Book.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd

Any value of n for an n-dimensional array is still a tensor. The word tensor is simply referring to the overall concept of an n-dimenional array.
We care about the shape of the tensor as we use different shapes to represent different types of data.

In [20]:
import torch

torch_scalar = torch.tensor(3.14)
torch_vector = torch.tensor([1, 2, 3, 4])
torch_matrix = torch.tensor([[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12] 
                            ])
torch_tensor3d = torch.tensor([ 
                               [[1, 2, 3, 4],
                                [5, 6, 7, 8],
                                [9, 10, 11, 12] 
                               ],
                               [[13, 14, 15, 16],
                                [17, 18, 19, 20],
                                [21, 22, 23, 24] 
                               ]
                              ])

print(torch_scalar.shape)
print(torch_vector.shape)
print(torch_matrix.shape)
print(torch_tensor3d.shape)

torch.Size([])
torch.Size([4])
torch.Size([3, 4])
torch.Size([2, 3, 4])
